# Sound Switch

# Experiment 1

## Methods

### Stimuli
- Each sound stimulus alternates between a C chord and a G chord strummed on guitar, taken from [here](https://www.apronus.com/music/onlineguitar.htm), which were subsequently recorded and amplified (25.98 db and 25.065 db respectively) using Audacity version 2.3.0 (using Effect --> Amplify) via Windows 10's Stereo Mix drivers.
- 9 switch rates between 0.1 and 0.9
- Each stimulus was 9,050 milliseconds long.
- 10 stimuli for each switch rate, for a total of 90 stimuli, all presented as a single block. 30 frame ITI.
- One practice stimulus at 0.5 switch rate presented at the start of the experiment.
- 20 repeated stimuli chosen via the same random choice for each subject (presentation order was via a unique random choice for each subject, however). Order can be found in the script.